In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import os

spark = SparkSession.builder \
    .appName("ManagedVsExternalTables") \
    .enableHiveSupport() \
    .getOrCreate()

data = [
    (1, "Jasprit Bumrah to Rohit Sharma, no run", "2023-01-01"),
    (2, "Trent Boult to Virat Kohli, FOUR runs", "2023-01-01"),
    (3, "Pat Cummins to Steve Smith, 1 run", "2023-01-02"),
    (4, "Rashid Khan to KL Rahul, SIX", "2023-01-02")
]
schema = StructType([
    StructField("ball_id", IntegerType(), False),
    StructField("commentary", StringType(), False),
    StructField("match_date", StringType(), False)
])
commentary_df = spark.createDataFrame(data, schema)

print("Creating managed table...")
commentary_df.write.mode("overwrite").saveAsTable("managed_commentary")
print("\nManaged table contents:")
spark.sql("SELECT * FROM managed_commentary").show()

external_location = "/tmp/external_tables/commentary_data"  
if os.path.exists(external_location):
    import shutil
    shutil.rmtree(external_location)

print("\nCreating external table...")
commentary_df.write.mode("overwrite") \
    .option("path", external_location) \

print("\nDropping managed table...")
spark.sql("DROP TABLE managed_commentary")

print("\nDropping external table...")
external_data_exists = os.path.exists(external_location)
print(f"Does external table data still exist after drop? {external_data_exists}")
print(f"Contents of external location: {os.listdir(external_location) if external_data_exists else 'None'}")

if external_data_exists:
    shutil.rmtree(external_location)

spark.stop()

Creating managed table...

Managed table contents:
+-------+--------------------+----------+
|ball_id|          commentary|match_date|
+-------+--------------------+----------+
|      1|Jasprit Bumrah to...|2023-01-01|
|      2|Trent Boult to Vi...|2023-01-01|
|      3|Pat Cummins to St...|2023-01-02|
|      4|Rashid Khan to KL...|2023-01-02|
+-------+--------------------+----------+


Creating external table...

Dropping managed table...

Dropping external table...
Does external table data still exist after drop? False
Contents of external location: None
